# This is the part 13 of the youtube ML course's code.

# Classification:

# First classification is K nearest neighbor(KNN)

## Import

In [41]:
import numpy as np
from sklearn import preprocessing, model_selection, neighbors, svm
import pandas as pd
import os
import file_operations

## Getting output directory and data directory

In [42]:
current_dir = os.getcwd()
print(current_dir)
paths = file_operations.create_project_path('Project 6 SVM')

d:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM
Project directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM'
Project data directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM\Data'
Project output directory created under 'd:\Important Files\Repositories\Machine-Learning-Project-related\Project 6 SVM\Output'


## Read File

In [43]:
file_name = 'breast-cancer-wisconsin.data'
file_path = os.path.join(paths["data_dir"], file_name)
df = pd.read_csv(file_path)
display(df)

,id,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_spith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


## For this file they have some notice made within breast-cancer-wisconsin.names file

7. Attribute Information: (class attribute has been moved to last column)

   #  Attribute                     Domain
   -- -----------------------------------------
   1. Sample code number            id number
   2. Clump Thickness               1 - 10
   3. Uniformity of Cell Size       1 - 10
   4. Uniformity of Cell Shape      1 - 10
   5. Marginal Adhesion             1 - 10
   6. Single Epithelial Cell Size   1 - 10
   7. Bare Nuclei                   1 - 10
   8. Bland Chromatin               1 - 10
   9. Normal Nucleoli               1 - 10
   10. Mitoses                       1 - 10
   11. Class:                        (2 for benign, 4 for malignant)

8. Missing attribute values: 16

   There are 16 instances in Groups 1 to 6 that contain a single missing 
   (i.e., unavailable) attribute value, now denoted by "?".  

9. Class distribution:
 
   Benign: 458 (65.5%)
   Malignant: 241 (34.5%)

## According to the 8 part: Missing attribute values, we have null values
## Processing NULL values:

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      699 non-null    int64 
 1   clump_thickness         699 non-null    int64 
 2   unif_cell_size          699 non-null    int64 
 3   unif_cell_shape         699 non-null    int64 
 4   marg_adhesion           699 non-null    int64 
 5   single_spith_cell_size  699 non-null    int64 
 6   bare_nuclei             699 non-null    object
 7   bland_chrom             699 non-null    int64 
 8   norm_nucleoli           699 non-null    int64 
 9   mitoses                 699 non-null    int64 
 10  class                   699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [45]:
df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
# df.replace('?', -99999, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 683 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      683 non-null    int64 
 1   clump_thickness         683 non-null    int64 
 2   unif_cell_size          683 non-null    int64 
 3   unif_cell_shape         683 non-null    int64 
 4   marg_adhesion           683 non-null    int64 
 5   single_spith_cell_size  683 non-null    int64 
 6   bare_nuclei             683 non-null    object
 7   bland_chrom             683 non-null    int64 
 8   norm_nucleoli           683 non-null    int64 
 9   mitoses                 683 non-null    int64 
 10  class                   683 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 64.0+ KB


You can see 16 null values within bare_nuclei column, as the description said.

## Data preprocessing

### Drop useless columns

In [46]:
df.drop(['id'], axis = 1, inplace = True)
display(df.head())

,clump_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_spith_cell_size,bare_nuclei,bland_chrom,norm_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


## Define X and y

In [47]:
X = np.array(df.drop(['class'], axis = 1))
y = np.array(df['class'])

## Split the train test.

In [48]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

## Defining classifier

In [49]:
clf = svm.SVC()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.9854014598540146


## Example predictions

In [50]:
example_measures = np.array([4, 2, 1, 1, 1, 2, 3, 2, 1]) 
example_measures = example_measures.reshape(1, -1)
prediction = clf.predict(example_measures)
print(prediction)

[2]
